## **SpDB Manual之系列5：数据绑定**

总述：IMAS将tokamak的装置藐视和物理概念进行了统一的分类、分层次管理，清晰描述了物理量之间的内在约束关系，形成了树状、层次化的、静态的IMAS本体。
数据绑定的目的，是将树状结构的IMAS本体的静态的表述转化为动态的实现。通过代码实现，将一系列静态数据，或者静态的物理量绑定在动态的实现类中，让数据赋予了行为，这样的话每个数据的变化会带动与之相关的数据的变化，这就实现物理量的自洽、自动演化。这既保证了物理量之间的关系的唯一确定性，又简化了计算模型的构建过程。

区别于传统的集成建模，传统的集成建模是将数据集成在一起，但是数据之间的关系是静态的，不会随着数据的变化而变化。而我们的方法是将数据和行为绑定在一起，赋予行为的数据是可操作的，同时带动与之关联的其他数据的变化。

- 普通字典
- 自定义数据类型
- 定义函数，自定义计算

实现：SpDB中提供API SpTree来实现数据绑定，它是一个装饰器，可以将一个类转化为SpTree类，同时给数据赋予更多的功能。
SpTree（）的操作对象：
- 内存中的Cache
- entry

In [24]:
### 初始化环境
from spdm.data.sp_property import sp_tree, sp_property, SpTree
from spdm.data.HTree import HTree, List, Dict
from spdm.data.Entry import open_entry
from spdm.data.AoS import AoS
import typing
import pprint

待定：

本来在裸数据里面是字典，没有功能，很多量也没有，特别是导出来

通过绑定，将导出来在需要的时候再计算，保证计算结果的一致性。关系是function保证的。保证结果的唯一性。



IMAS 标称属性转化为。。。。

按照属性访问
类型提示，将裸数据转化为具有功能的对象实体（数组转化为表达式或者function）

#### 原生Python中字典的访问(树状结构的访问)

In [117]:
### 原生Python中字典的访问(树状结构的访问)
data = {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    }
    # _entry=open_entry(....)

In [68]:
data["name"]


'Alice'

In [69]:
data["age"]

25

In [118]:
data["address"]

[{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'},
 {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
 {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]

In [89]:
data["address"][0]

{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'}

#### 将Python中原生数据映射到SpTree中

In [119]:
### 利用sp_tree装饰器定义Data类，Data的输入是一个字典，输出是一个树状结构
@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
### 传递一个普通的字典结构给Data
data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)

In [91]:
## 访问字典中的数据
data.get("name")

'Alice'

In [120]:
data.get("address")

[{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'},
 {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
 {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]

In [96]:
### 但是address不支持直接切片访问。
data.get("address[1]")

<tags.not_found: 0>

In [121]:
### address只能以整体拿回来，然后再切片访问
data.get("address")[1]

{'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'}

#### sp_tree装饰器支持在Data类中声明字典中的每个元素key及其value的数据类型

In [122]:
### sp_tree装饰器支持在Data类中声明字典中的每个元素key及其value的数据类型，这样就可以直接访问字典中的元素。
@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    name: str
    age: int
    hobbies: List[str]
    address: List[Dict]   
    # address: AoS[Dict]
    # address: AoS[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": '25',
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)

In [108]:
## name已经在Class Data() 中声明
data.name

'Alice'

In [111]:
### 除了声明元素的key值，同时可以强制转化原有数据的数据类型
type(data.age)

int

In [123]:
### 数据在_cache里面
data.address._cache

[{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'},
 {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
 {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]

In [62]:
## zip没有被声明
data.address[0].zip

AttributeError: 'Dict' object has no attribute 'zip'

In [112]:
### 支持对LIST元素的直接切片访问，而不需要拿回整个字典
data.address[0]._cache

{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'}

### 进一步定义复杂AOS内的数据结构

In [129]:
### 增加Address类，定义address的数据类型

@sp_tree
class Addrees:
    street: str
    city: str
    state: str
    zip: int


@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    name: str
    age: int
    hobbies: List[str]
    address: AoS[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": '25',
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)

In [130]:
### zip的颜色变成了蓝色，因为在Addrees类中声明了zip
data.address[0].zip

12345

In [132]:
data.address[1].street

'456 Oak St'

In [131]:
data.address[0]._cache

{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': 12345}

### 直接在Class Adress中增加元素
- 增加building 和neighbour
- 可以设置默认值：building: str = "#12345"


In [134]:


@sp_tree
class Addrees:
    street: str
    city: str
    state: str
    zip: int
    building: str = "#12345"
    neighbour: str


@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    name: str
    age: int
    hobbies: List[str]
    address: AoS[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
)



In [135]:
data.address[0].building

'#12345'

In [137]:
data.address[0].neighbour="zhangshan"

In [144]:
### 应用才会生效
data.address[1].building

'#12345'

In [139]:
data.address[0].neighbour

'zhangshan'

In [146]:
data.address[1].neighbour = "lisi"

In [142]:
data.address[0]._cache

{'street': '123 Main St',
 'city': 'Anytown',
 'state': 'CA',
 'zip': '12345',
 'building': '#12345',
 'neighbour': 'zhangshan'}

In [147]:
data.address[1]._cache

{'street': '456 Oak St',
 'city': 'Othertown',
 'state': 'NY',
 'zip': '67890',
 'building': '#12345',
 'neighbour': 'lisi'}

### 高阶功能，增加函数sp_property，定义函数

In [149]:
@sp_tree
class House:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        pprint.pprint((args, kwargs))

    level: int = 4
    length: float
    width: float

    @sp_property(units="m^2")
    def area(self) -> float:
        return self.width * self.length


@sp_tree
class Addrees:
    street: str
    city: str
    state: str
    zip: int
    building: str = "#12345"
    neighbour: str
    house: House = sp_property(label="big house", default_value={"level": 2, "area": 1000})


@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    name: str
    age: int
    hobbies: List[str]
    address: AoS[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
                "house": {"length": 5, "width": 10},
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
)



In [150]:
house=data.address[1].house

(({'length': 5, 'width': 10},),
 {'_entry': None,
  '_parent': <__main__.Addrees object at 0x7efc7ea7f760>,
  'default_value': {'area': 1000, 'level': 2},
  'label': 'big house',
  'name': 'house'})


In [151]:
house.level

4

In [152]:
house.length

5.0

In [153]:
house.width

10.0

In [154]:
house.area

50.0

In [155]:
House.area.metadata.get("units")

'm^2'

In [156]:
data.get("address/3/street", "I don't know")

"I don't know"

In [157]:
for address in data.address:
    print(address.street)

123 Main St
456 Oak St
789 Elm St
tags.not_found


In [158]:
data.address[0].street = "456 Main St"

In [159]:
data.address[0].street

'456 Main St'

In [ ]:
data2 = Data({}, _entry=open_entry("file://./data/address.json"))

In [ ]:

file_entry可以给data，这个明天试下。
file_entry = open_entry(f"file+geqdsk://{workdir}//data/g900003.00230_ITER_15MA_eqdsk16HR.txt/#equilibrium")

In [2]:



class Foo(SpTree):
    boo: Data = sp_property(label="boo", default_value=1.0)


class Bar(Foo):
    boo: Data = sp_property(label="boo", units="m")


@sp_tree
class Bar2(Foo):
    boo: Data = 2.1234


a = Bar()
b = Bar2()
print(a.boo._metadata)
print(b.boo._metadata)
print(b.get("boo")._metadata)

{'label': 'boo', 'name': 'boo', 'units': 'm'}
{'label': 'boo', 'name': 'boo'}
{'label': 'boo', 'name': 'boo'}
